In [139]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table, event
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.orm import sessionmaker
from sqlalchemy import join
from sqlalchemy import select

from sqlalchemy.orm import registry

import random
import string

In [140]:
#calling sqlalchemy declerative base for models
Base = declarative_base()

<ipython-input-140-14385c17bab9>:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [141]:
#creating models with declerative mapping
class User(Base):
    __tablename__ = 'users'
    __table_args__ = {'extend_existing': True}


    id = Column(Integer, primary_key=True)
    username = Column(String, unique=True, nullable=False)
    email = Column(String, unique=True, nullable=False)

    comments = relationship('Comment', back_populates='user')

    @property #py descriptors
    def full_email(self):
      return f"{self.username} <{self.email}>"
    @property
    def username_length(self):
      return len(self.username)

    def __repr__(self):
      return f"<User(id={self.id}, username = '{self.username}', email = '{self.email}')>"

@event.listens_for(User, "load") # event hooks
def receive_load_user(target,context):
  print("User loaded from Database:",target)

@event.listens_for(User, "refresh")
def receive_refersh_user(target,context,attrs=None):
  print("User refreshed:", target)




class Comment(Base):
    __tablename__ = 'comments'
    __table_args__ = {'extend_existing': True}


    id = Column(Integer, primary_key=True)
    text = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))

    user = relationship('User', back_populates='comments')



In [142]:
#create an engine for e.g: sqlite ... db connection
engine = create_engine('sqlite:///example.db')

In [143]:
#Base.metadata.create_all(engine) creates all the tables defined in the Base
#class in the database specified by the engine.
# In this case, it will create the 'users' and 'comments' tables in the 'example.db' SQLite database.

Base.metadata.create_all(engine)

In [144]:
#binding session
Session = sessionmaker(bind=engine)
session = Session()

In [145]:
#persisting the db
try:
    for i in range(1, 11):
        email = f'user{i}' + ''.join(random.choices(string.ascii_lowercase, k=5)) + '@example.com'
        user = User(username=f'user{i}', email=email)
        session.add(user)

    session.commit()

    for user in session.query(User).all():
        for i in range(1, 4):
            comment = Comment(text=f'Comment {i} by {user.username}', user_id=user.id)
            session.add(comment)

    session.commit()

except Exception as e:
    session.rollback()
    print(f"An error occurred: {e}")

finally:
    session.close()

An error occurred: (sqlite3.IntegrityError) UNIQUE constraint failed: users.username
[SQL: INSERT INTO users (username, email) VALUES (?, ?) RETURNING id]
[parameters: ('user1', 'user1ctlyl@example.com')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [146]:
#simple select users
users = session.query(User).all()
for user in users:
    print(f"User ID: {user.id}, Username: {user.username}, Email: {user.email}, Comments: {len(user.comments)}")

User loaded from Database: <User(id=1, username = 'user1', email = 'user1pweeh@example.com')>
User loaded from Database: <User(id=2, username = 'user2', email = 'user2uzabp@example.com')>
User loaded from Database: <User(id=3, username = 'user3', email = 'user3kqptl@example.com')>
User loaded from Database: <User(id=4, username = 'user4', email = 'user4dimph@example.com')>
User loaded from Database: <User(id=5, username = 'user5', email = 'user5htdrh@example.com')>
User loaded from Database: <User(id=6, username = 'user6', email = 'user6unqmz@example.com')>
User loaded from Database: <User(id=7, username = 'user7', email = 'user7darbi@example.com')>
User loaded from Database: <User(id=8, username = 'user8', email = 'user8dusro@example.com')>
User loaded from Database: <User(id=9, username = 'user9', email = 'user9oqctv@example.com')>
User loaded from Database: <User(id=10, username = 'user10', email = 'user10adihe@example.com')>
User loaded from Database: <User(id=11, username = 'newte

In [107]:
#simple select comments
comments = session.query(Comment).all()
for comment in comments:
    print(f"Comment ID: {comment.id}, Text: {comment.text}, User ID: {comment.user_id}")

Comment ID: 1, Text: Comment 1 by user1, User ID: 1
Comment ID: 2, Text: Comment 2 by user1, User ID: 1
Comment ID: 3, Text: Comment 3 by user1, User ID: 1
Comment ID: 4, Text: Comment 1 by user2, User ID: 2
Comment ID: 5, Text: Comment 2 by user2, User ID: 2
Comment ID: 6, Text: Comment 3 by user2, User ID: 2
Comment ID: 7, Text: Comment 1 by user3, User ID: 3
Comment ID: 8, Text: Comment 2 by user3, User ID: 3
Comment ID: 9, Text: Comment 3 by user3, User ID: 3
Comment ID: 10, Text: Comment 1 by user4, User ID: 4
Comment ID: 11, Text: Comment 2 by user4, User ID: 4
Comment ID: 12, Text: Comment 3 by user4, User ID: 4
Comment ID: 13, Text: Comment 1 by user5, User ID: 5
Comment ID: 14, Text: Comment 2 by user5, User ID: 5
Comment ID: 15, Text: Comment 3 by user5, User ID: 5
Comment ID: 16, Text: Comment 1 by user6, User ID: 6
Comment ID: 17, Text: Comment 2 by user6, User ID: 6
Comment ID: 18, Text: Comment 3 by user6, User ID: 6
Comment ID: 19, Text: Comment 1 by user7, User ID: 7
Co

In [108]:
#select with join
comments_with_users = session.query(Comment, User).join(User).all()
for comment, user in comments_with_users:
    print(f"Comment ID: {comment.id}, Text: {comment.text}, User ID: {user.id}, Username: {user.username}, Email: {user.email}")


Comment ID: 1, Text: Comment 1 by user1, User ID: 1, Username: user1, Email: user1pweeh@example.com
Comment ID: 2, Text: Comment 2 by user1, User ID: 1, Username: user1, Email: user1pweeh@example.com
Comment ID: 3, Text: Comment 3 by user1, User ID: 1, Username: user1, Email: user1pweeh@example.com
Comment ID: 4, Text: Comment 1 by user2, User ID: 2, Username: user2, Email: user2uzabp@example.com
Comment ID: 5, Text: Comment 2 by user2, User ID: 2, Username: user2, Email: user2uzabp@example.com
Comment ID: 6, Text: Comment 3 by user2, User ID: 2, Username: user2, Email: user2uzabp@example.com
Comment ID: 7, Text: Comment 1 by user3, User ID: 3, Username: user3, Email: user3kqptl@example.com
Comment ID: 8, Text: Comment 2 by user3, User ID: 3, Username: user3, Email: user3kqptl@example.com
Comment ID: 9, Text: Comment 3 by user3, User ID: 3, Username: user3, Email: user3kqptl@example.com
Comment ID: 10, Text: Comment 1 by user4, User ID: 4, Username: user4, Email: user4dimph@example.com

In [109]:
#add new user
try:
    new_user = User(username='test_user', email='test_user1@example.com')
    session.add(new_user)
    session.commit()

    new_comment = Comment(text='This is a new comment testing.', user_id=new_user.id)
    session.add(new_comment)
    session.commit()

    user_with_comment = session.query(User, Comment).join(Comment).filter(User.id == new_user.id).first()
    user, comment = user_with_comment
    print(f"User ID: {user.id}, Username: {user.username}, Email: {user.email}")
    print(f"Comment ID: {comment.id}, Text: {comment.text}")

except Exception as e:
    session.rollback()
    print(f"An error occurred: {e}")

finally:
    session.close()

An error occurred: (sqlite3.IntegrityError) UNIQUE constraint failed: users.email
[SQL: INSERT INTO users (username, email) VALUES (?, ?)]
[parameters: ('test_user', 'test_user1@example.com')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [110]:
#update user
try:
    user_to_update = session.query(User).filter(User.username == 'test_user').first()

    if user_to_update:
        user_to_update.username = 'newtest1_user'
        session.commit()
        print("Username updated successfully!")
    else:
        print("User not found!")

except Exception as e:
    session.rollback()
    print(f"An error occurred: {e}")

finally:
    session.close()

User not found!


In [111]:
name = 'user1'
try:
    stmt_user = select(User).where(User.username == name)
    example_user = session.execute(stmt_user).scalar()

    if example_user:
        new_comment = Comment(text="This is a new comment for make_change_doc.", user_id=example_user.id)
        session.add(new_comment)
        session.commit()
        print("Changes committed successfully for "+name)
    else:
        print("User  name not found."+name)

except Exception as e:
    session.rollback()
    print(f"An error occurred: {e}")

finally:
    session.close()


Changes committed successfully for user1


In [112]:
#imperative mapping for comments and user tables:


mapper_registry = registry()

user_table = Table(
    "user_imp",
    mapper_registry.metadata,
    Column("id",Integer,primary_key = True),
    Column("username", String(50),unique = True, nullable = False),
    Column("email", String(50), nullable = False),
    )

comments_table = Table(
    "comments_imp",
    mapper_registry.metadata,
    Column("id",Integer,primary_key = True),
    Column("text", String(100), nullable = False),
    Column("user_id",Integer, ForeignKey("user_imp.id")), #fr key defination
    )

class User_imp:
  pass

class Comments_imp:
  pass

mapper_registry.map_imperatively(User_imp, user_table)


mapper_registry.map_imperatively(Comments_imp, comments_table, properties = {
    "user": relationship(User_imp)

})



<Mapper at 0x7ce502264c10; Comments_imp>

In [129]:
# Delete all entries from the tables so we can repopulate in next cell
session.query(User_imp).delete()
session.query(Comments_imp).delete()

# Commit deletion
session.commit()

# Drop tables for recreation of tables in next cell
mapper_registry.metadata.drop_all(engine)


In [130]:
#test population of the db in existing egine and session

mapper_registry.metadata.create_all(engine) # Create the tables


user1 = User_imp()
user1.username = 'user3'
user1.email = 'user3@example.com'

user2 = User_imp()
user2.username = 'user4'
user2.email = 'user4@example.com'

comment1 = Comments_imp()
comment1.text = 'Comment 3'
comment1.user = user1

comment2 = Comments_imp()
comment2.text = 'Comment 4'
comment2.user = user2

session.add(user1)
session.add(user2)
session.add(comment1)
session.add(comment2)

session.commit()#commit insertion

<ipython-input-130-dfbeeabfbc9f>:27: SAWarning: Identity map already had an identity for (<class '__main__.Comments_imp'>, (1,), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  session.commit()#commit insertion
<ipython-input-130-dfbeeabfbc9f>:27: SAWarning: Identity map already had an identity for (<class '__main__.Comments_imp'>, (2,), None), replacing it with newly flushed object.   Are there load operations occurring inside of an event handler within the flush?
  session.commit()#commit insertion


In [131]:
u1 = session.scalars(select(User_imp).where(User_imp.username == "user4")).first()
#select(User_imp).where(User_imp.username == "user4")


if u1:
    print("User found:")
    print(f"Username: {u1.username}")
    print(f"Email: {u1.email}")
else:
    print("User not found")

User found:
Username: user4
Email: user4@example.com


In [135]:
all_user_imp= session.query(User_imp).all()


for user in all_user_imp:
    print(user.username)

user3
user4


In [136]:
u1_del = session.scalars(select(User_imp).where(User_imp.username == "user4")).first()

if u1_del:
    print("User found:")
    print(f"Username: {u1_del.username}")
    print(f"Email: {u1_del.email}")

    session.delete(u1_del)
    session.commit()
    print("User deleted successfully")
else:
    print("User not found")

User found:
Username: user4
Email: user4@example.com
User deleted successfully


In [ ]:
#Runtime Introspection of Mapped classes, Instances and Mappers(Working)